## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import heapq
import mplcursors
from tqdm import tqdm

## Heap

In [ ]:
# Use a max heap of n elements, popping when the new element is less than the head
class KMaxHeap():
  def __init__(self, capacity):
    self.capacity = capacity
    self.data = [] 

  def push(self, elem):
    elem[0] = -elem[0]
    if len(self.data) < self.capacity:
      heapq.heappush(self.data, elem)
    elif elem[0] > self.data[0][0]:
      heapq.heappushpop(self.data, elem)

## BirdFlockSimulator

In [ ]:
from numpy import disp


# FIXME Approx K
class NearestNeighbourCalculator:
  
  def __init__(self, neighboursN):
    self.neighboursN = neighboursN
    # FIXME
    self.predN = 1
  
  def getPredNNDf(self, predDf, preyDf):
    res = []
    # FIXME Get from cache
    for i in range(len(predDf)):
      res.append(self.getNearest(predDf.iloc[i], preyDf, 2))
    predNNdf = pd.DataFrame(np.array(res))
    predNNdf.rename(mapper=self.mapPredColNames, axis='columns', inplace=True)
    ret = pd.concat([predDf, predNNdf], axis='columns')
    # Drop cols from the orig predator DF
    return ret.drop(columns=['x', 'y', 'vx', 'vy'])

  def getPreyNNDf(self, preyDf, predDf):
    # Prey
    preyRes = []
    for i in range(len(preyDf)):
      preyRes.append(self.getNearest(preyDf.iloc[i], preyDf, 4, i))
    preyNNdf = pd.DataFrame(np.array(preyRes))
    preyNNdf.rename(mapper=self.mapPreyColNames, axis='columns', inplace=True)

    # Pred
    predRes = []
    for i in range(len(preyDf)):
      predRes.append(self.getNearest(preyDf.iloc[i], predDf, 2))
    predNNdf = pd.DataFrame(np.array(predRes))
    predNNdf.rename(mapper=self.mapPredColNames, axis='columns', inplace=True)
  
    ret = pd.concat([preyDf, preyNNdf, predNNdf], axis='columns')
    # Drop cols from the orig prey DF
    return ret.drop(columns=['x', 'y', 'vx', 'vy'])

  # Nearest Neighbours
  def getNearest(self, target, df, numData, ignoreIndex=None):
    # Get the nearest N Prey
    preyHeap = KMaxHeap(self.neighboursN)
    for index, row in df.iterrows():
      if ignoreIndex is not None and index == ignoreIndex:
        continue
      dist = abs(row.x-target.x) ** 2 + abs(row.y-target.y) ** 2
      preyHeap.push([dist, row])
    preyHeap.data.sort()

    # Return all NN
    nn = np.array(
      # Dist + X, Y, Vx, Vy
      [[math.sqrt(-neighbour[0])] + [neighbour[1][j] for j in range(numData)]
      for neighbour in reversed(preyHeap.data)]).flatten()
    return nn

  def mapPredColNames(self, col):
    predn = col//3
    label = ""
    if col % 3 == 0:
      label = "dist"
    elif col % 3 == 1:
      label = "x"
    elif col % 3 == 2:
      label = "y"
    return f"pred_{predn}_{label}"

  def mapPreyColNames(self, col):
    nnn = col//5
    label = ""
    if col % 5 == 0:
      label = "dist"
    elif col % 5 == 1:
      label = "x"
    elif col % 5 == 2:
      label = "y"
    elif col % 5 == 3:
      label = "vx"
    else:
      label = "vy"
    return f"nn_{nnn}_{label}"
  

In [ ]:
class PreyVectorCalculator:
  def __init__(self, neighBoursN, predFactor, momentumFactor, boundaryFactor, maxX, maxY):
    self.neighboursN = neighBoursN
    # TODO
    self.predNeighboursN = 1
    self.predFactor = predFactor
    self.momentumFactor = momentumFactor
    self.boundaryFactor = boundaryFactor
    self.maxX = maxX
    self.maxY = maxY

  def getBoundaryVectors(self, row):
    x = 0.0
    y = 0.0
    # X Axis
    if row['x'] < 10:
      x = 10-row['x']
    if row['x'] > self.maxX - 10:
      x = (self.maxX - 10) - row['x']
    # Y Axis
    if row['y'] < 10:
      y = 10-row['y']
    if row['y'] > self.maxY - 10:
      y = (self.maxY - 10) - row['y']
    return x*self.boundaryFactor ,y*self.boundaryFactor

  def calcVectors(self, preyRow):
    preyXVectors = []
    preyYVectors = []
    predXVectors = []
    predYVectors = []
    vectorReason = []
    # Get Vectors from neighbouring prey
    for i in range(self.neighboursN):
      xDiff, yDiff, reason = self.calcPreyVector(preyRow, self.neighboursN, i)
      preyXVectors.append(float(xDiff))
      preyYVectors.append(float(yDiff))
      vectorReason.append(reason)

    # Add vectors for hitting boundary
    xBoundary, yBoundary = self.getBoundaryVectors(preyRow)

    # Get vectors from neighbouring predators
    for i in range(self.predNeighboursN):
      xDiff, yDiff = self.calcPredVector(preyRow, self.predNeighboursN, i)
      predXVectors.append(float(xDiff))
      predYVectors.append(float(yDiff))
    

    return preyXVectors + [xBoundary] + preyYVectors + [yBoundary] + vectorReason + predXVectors + predYVectors

  # Vector calculation
  def getVectors(self, df, prev):
    ret = pd.concat([df, prev], axis=1).apply(self.calcVectors, axis=1, result_type='expand')
    preyXVectorCols = [f'vx_{i}' for i in range(self.neighboursN)] + ['vx_b']
    preyYVectorCols = [f'vy_{i}' for i in range(self.neighboursN)] + ['vy_b']
    reasonCols = [f'r_{i}' for i in range(self.neighboursN)]
    predXVectorCols = [f'pred_vx_{i}' for i in range(self.predNeighboursN)]
    predYVectorCols = [f'pred_vy_{i}' for i in range(self.predNeighboursN)]
    ret.columns = preyXVectorCols + preyYVectorCols + reasonCols + predXVectorCols + predYVectorCols
    ret['vx_d'] = ret[preyXVectorCols + predXVectorCols].sum(axis=1)
    ret['vy_d'] = ret[preyYVectorCols + predYVectorCols].sum(axis=1)
    # Maintain momentum from previous velocity
    ret['x'] = ret['vx_d'] * (1-self.momentumFactor) + prev.vx * (self.momentumFactor)
    ret['y'] = ret['vy_d'] * (1-self.momentumFactor) + prev.vy * (self.momentumFactor)
    # Min speed
    # Prob a more efficient way of doing this
    for index, row in ret.iterrows():
      if ((row.x **2) + (row.y ** 2)) < 10:
        newX, newY = self.normalizeVector(row.x, row.y, math.sqrt(10))
        ret.iat[index, ret.columns.get_loc('x')] = newX
        ret.iat[index, ret.columns.get_loc('y')] = newY
    return ret

  def calcPredVector(self, row, N, i):
    predX = row[f"pred_{i}_x"]
    predY = row[f"pred_{i}_y"]
    predDist = row[f"pred_{i}_dist"]
    
    vecX = row.x-predX
    vecY = row.y-predY

    repulsion = (1/predDist) * self.predFactor
    
    x,y = self.normalizeVector(vecX, vecY, repulsion / N)
    return x, y

  def calcPreyVector(self, row, N, i):
    neighbourX = row[f"nn_{i}_x"]
    neighbourY = row[f"nn_{i}_y"]
    neighbourDist = row[f"nn_{i}_dist"]
    neighbourVx = row[f"nn_{i}_vx"]
    neighbourVy = row[f"nn_{i}_vy"]

    vecX = neighbourX-row.x
    vecY = neighbourY-row.y
    # If neighbour is greater than 11, we want to be attracted to a point 10 away
    if neighbourDist >= 15:
      attraction = neighbourDist-10
      attractionFactor = 0.1
      x,y = self.normalizeVector(vecX, vecY, attraction*attractionFactor/N)
      return (x, y, "A")
    # If neighbour is less than 9, repulse them based on how close they are
    # Push away linearly
    elif neighbourDist <= 5:
      repulsion = 10-neighbourDist
      repulsionFactor = 0.1
      x,y = self.normalizeVector(-vecX, -vecY, repulsion*repulsionFactor/N)
      return (x, y, "R")
    # Else, copy vector of the neighbour
    else:
      return (neighbourVx/N, neighbourVy/N, "C")
  
  def normalizeVector(self, x, y, scale=1):
    hyp = math.sqrt(x**2 + y**2)
    return ((x/hyp)*scale, (y/hyp)*scale)

In [ ]:
class PredatorVectorCalculator:
  def __init__(self, maxSpeed):
    self.maxSpeed = maxSpeed

  def getVectors(self, predators, nndf):
    # TODO
    ret = predators.copy()
    ret.x = ((nndf.pred_0_x - ret.x) / nndf.pred_0_dist) * math.sqrt(self.maxSpeed)
    ret.y = ((nndf.pred_0_y - ret.y) / nndf.pred_0_dist) * math.sqrt(self.maxSpeed)
    return ret[['x', 'y']]

In [ ]:
class BirdFlockSimulator:

  # Init
  def __init__(self, numBirds, seed):
    self.numBirds = numBirds
    self.seed = seed
    self.maxX = 500
    self.maxY = 500
    self.neighboursN = 5
    self.momentumFactor = 0.5
    self.boundaryFactor = 0.5
    self.predFactor = 50
    self.maxPredSpeed = 10
    np.random.seed(self.seed)
    self.birds = self.createXYObjectsDataframe(self.numBirds)
    self.predators = self.createXYObjectsDataframe(1)
    self.next = None

  def createXYObjectsDataframe(self, NObjects):
    d = {
      'x': np.random.rand(NObjects) * self.maxX,
      'y': np.random.rand(NObjects) * self.maxY,
      'vx' : np.random.rand(NObjects) * 10,
      'vy' : np.random.rand(NObjects) * 10
    }
    return pd.DataFrame(data=d)

  def getNext(self):
    if self.next is None:
      self.next = self.calculateTick()
    return self.next[0]

  def tick(self, update=True):
    self.getNext()
    if update:
      self.birds, self.nnDf, self.predators, self.nnDfPred = self.next
      self.next = None

  def calculateTick(self):
    # NN
    nnCalc = NearestNeighbourCalculator(self.neighboursN)
    nnDfPrey = nnCalc.getPreyNNDf(self.birds, self.predators)

    # Calculate Prey Vectors
    preyCalc = PreyVectorCalculator(self.neighboursN, self.predFactor, self.momentumFactor, self.boundaryFactor, self.maxX, self.maxY)
    preyVectors = preyCalc.getVectors(nnDfPrey, self.birds)
    # Update Prey Vectors
    newD = self.birds[['x', 'y']] + preyVectors[['x', 'y']]
    preyVectors = preyVectors.rename(columns={'x' : 'vx', 'y' : 'vy'})
    ret = pd.concat([newD, preyVectors], axis=1)

    # NN
    nnDfPred = nnCalc.getPredNNDf(self.predators, self.birds)

    # Calculate Pred Vectors
    predCalc = PredatorVectorCalculator(self.maxPredSpeed)
    predVectors = predCalc.getVectors(self.predators, nnDfPred)
    # Update Predator Vectors
    newPred = self.predators[['x', 'y']] + predVectors[['x', 'y']]
    predVectors = predVectors.rename(columns={'x' : 'vx', 'y' : 'vy'})
    predRet = pd.concat([newPred, predVectors], axis=1)
    return ret, nnDfPrey, predRet, nnDfPred

  

## Plotting

In [ ]:
%matplotlib widget

def selFun(sel):
  sel.annotation.set_text(sel.artist.annotation_names[sel.target.index])

def positionPlot(curr):
  fig, ax = plt.subplots(1, 1)
  scat = ax.scatter(curr["x"], curr["y"])
  scat.annotation_names = list(range(100))
  cursor = mplcursors.cursor([scat], hover=True)
  cursor.connect("add", lambda sel: selFun(sel))
  plt.show()

In [ ]:
def velPlot(sim):
  # Get Data
  curr = sim.birds
  next = sim.getNext()
  # Plot data
  fig, ax = plt.subplots()
  cxs = list(curr.x)
  cys = list(curr.y)
  nxs = list(next.x)
  nys = list(next.y)
  for cx, cy, nx, ny in zip(cxs, cys, nxs, nys):
    ax.annotate('', xytext=(cx, cy), xy=(nx, ny), arrowprops=dict(arrowstyle='->'))
  ax.set(xlim=(-10, 510), ylim=(-10, 510))
  plt.close(fig)
  return fig

In [ ]:
def posPlot(sim):
  fig, ax = plt.subplots(1, 1)

  # Plot Prey
  preyData = sim.birds
  preyScatter = ax.scatter(preyData["x"], preyData["y"])
  preyScatter.annotation_names = list(range(100))

  predData = sim.predators
  predScatter = ax.scatter(predData['x'], predData['y'], color='red')
  cursor = mplcursors.cursor([preyScatter], hover=True)
  cursor.connect("add", lambda sel: selFun(sel))

  ax.set(xlim=(-10, 550), ylim=(-10, 550))
  plt.close(fig)
  return fig

## Test

In [ ]:
sim = BirdFlockSimulator(100, 1)

In [ ]:
sim.tick()

## Position Plot


In [ ]:
for i in tqdm(range(500)):
  sim.tick()
  fig = posPlot(sim)
  num = str(i).rjust(3, '0')
  fig.savefig(f'figs/{num}.png')